# Colombia J&J

J&J Calidad de operativos <br>
País: Colombia <br>
Nombre: Colombia <br>
Autor: Juan José Rincón<br>
Fecha de creación: 16-06-2022<br>
Fecha Modificación: 17-06-2022<br>

In [1]:
#!pip install os numpy pandas
#!pip install openpyxl
import os
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from sklearn import preprocessing

Establece las variables generales y paths

In [2]:
os.chdir("/Users/JuanJose/Library/CloudStorage/OneDrive-UniversidaddelosAndes/Econometria/J&J/100652_J_J/Colombia/Stata")
Path_Input_xlsx = "DataFiles/Input/XLS/"
Path_working = "DataFiles/Working/"
Path_out_xlsx = "DataFiles/Output/XLS/"
#_date=datetime.datetime(2021, 12,1)
_Date=datetime(2021,12,1)+relativedelta(months=1)-timedelta(days=1)
#_Date=_Date.strftime("%d/%m/%Y")
_date=date.today()
_date=_date.strftime("%d/%m/%Y")

#### Importación de datos
Se importan los datos y se crea un diccionario con los labels de las variables

In [3]:
_colombia=Path_Input_xlsx+'Col_dic21.xlsx'
Colombia=pd.read_excel(_colombia,sheet_name='pacientes')
#colombia_labels= {column: column for column in Colombia.columns}

##### Filtrado de datos no validos
Algunos que no tienen id, que fueron enviados por miembros de Econometria como pruebas, o que sucedieron antes de octubre de 2014

In [4]:
Colombia['date_surgery']=pd.to_datetime(Colombia['date_surgery'])
Colombia.drop(Colombia[Colombia['Grupo']=='Econometria'].index, inplace = True)
Colombia.drop(Colombia[Colombia['id'].isnull()].index, inplace = True)
Colombia.drop(Colombia[Colombia['id'].isna()].index, inplace = True)
Colombia.drop(Colombia[Colombia['id']==0].index, inplace = True)
Colombia.drop(Colombia[Colombia['date_surgery']< datetime(2014,10,1)].index, inplace = True)

In [5]:
Variables=Colombia.columns.to_list()

Se crea una variable de dias transcurridos desde la cirugía

In [6]:
Colombia['Dias']=_Date-Colombia['date_surgery']

Se reemplazan los valores para convertir en string

In [7]:
Colombia=Colombia.replace(["No","si","NS/NR","Fem","Masc",'Sí'], [0,1,9,1,2,1])

# Renombrar variables 
se renombran algunas variables

In [8]:
vars={'identification':'documento', 'height':'Talla', 'weight':'PesoInicial', 'malnutrition':'Malnutricion', 'age_at_surgery':'EdadCiru','funding':'Financiacion', 
'gender':'Sexo', 'previous_surgery':'ProcePrev','technique_used':'TecnicaU', 'date_surgery':'FechaCirugia',
'surgery_type':'TipoCB', 'bypass_used':'Tbypass','other_procedures':'OtrosProce','other_procedures_other':'OtroProceCual','type_identification':'tipodoc',
'colombian_resident':'ResideCol', 'adjustable_gastric_band':'BandaGaA', 'no_adjustable_gastric_band':'BandaGaNA', 'bypass_roux':'BypassYR',
'bypass_mini':'MiniGaBypass', 'sleeve_gastrectomy':'GaManga', 'duodenal_crossing':'CruceDuodenal', 'sadi':'SADIS', 'scopinaro':'DeBilioScopi', 
'gastric_plicature':'PliGa', 'gastroplasty':'GaVcB', 'previous_surgery_other':'Otra', 'previous_surgery_other_which':'Cual', 'gastric_ballon':'BalGas',
'hospital_departure':'fecha_salidaH', 'other_procedures_legacy':'OtrosProcelegacy', 'conversion_cause_legacy':'CausaConverLegacy', 'weight_gain':'GananciaPeso',
'reflux':'Reflujo', 'hypoglycemia':'Hipoglicemia', 'type_conversion_surgery':'TipoCirugiaConver', 'cholecystectomy':'Colecistec',
'hiatus_correction':'CorrecHiato','liver_biopsy':'BiopsiaHep','no_other_procedures':'NsrO'} 
Colombia.rename(columns=vars,inplace=True)

## Creación y organización de variables

Se crea la función Reorder columns, para reordenar las columnas según la posición deseada (detrás de cierta variable indicada)

In [9]:
#funcion para ordenar variables
def reorder_columns(dataframe, col_names, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string list): columns names to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    Col_names=list()
    Col_names.extend(col_names)
    col_names=[]
    col_names.extend(Col_names)
    n=len(col_names)
    
    for i in range(n-1,-1,-1):
        col_name = col_names[i]
        temp_col = dataframe[col_name]
        dataframe = dataframe.drop(columns=[col_name])
        dataframe.insert(loc=position, column=col_name, value=temp_col)

    return dataframe

In [10]:
#tipo doc
Colombia["tipodoc"] = Colombia["tipodoc"].astype('category')
#Para encontrar la codificacion numerica de la variable categorica se usa .cat.codes -> Colombia["tipodoc"] = Colombia["tipodoc"].cat.codes 

In [14]:
Colombia["Sexo"] = Colombia["Sexo"].astype('category')
position=Colombia.columns.to_list().index('Sexo')+1
pd.get_dummies(Colombia['Sexo'],prefix='Sexo',columns=['Mujer','Hombre'])

,Sexo_1,Sexo_2
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
10326,0,1
10327,1,0
10328,1,0
10329,1,0


#### Sexo
Se crean variables para cada uno de los sexos 

In [ ]:
position=Colombia.columns.to_list().index('Sexo')+1
Colombia.insert(position,'Sexo_Hombre',np.where(Colombia['Sexo'] == 2, 1, 0))
Colombia.insert(position,'Sexo_Mujer',np.where(Colombia['Sexo'] == 1, 1, 0))

#### Edad
Se crean variables de edad por rangos y edad por sexo 

In [ ]:
#rangos de edad
bins = [0,15, 25, 35, 45, 55, 120]
labels = ['-15','15-25', '25-35', '35-45', '45-55','55+']
Colombia['RangoEdad'] = pd.cut(Colombia['EdadCiru'], bins, labels = labels,include_lowest = True) # al poner labels la variable queda tipo str
position=Colombia.columns.to_list().index('EdadCiru')+1
Colombia=reorder_columns(Colombia,['RangoEdad'], position)
del bins, labels

In [ ]:
# edad por sexo
Colombia['EdM']=Colombia['EdadCiru']*Colombia['Sexo_Mujer']
Colombia['EdH']=Colombia['EdadCiru']*Colombia['Sexo_Hombre']
position=Colombia.columns.to_list().index('RangoEdad')+1
Colombia=reorder_columns(Colombia, ['EdM','EdH'], position)

#### Peso
Se crean variable PesoIdeal y ExcesoPeso

In [ ]:
Colombia['PesoIdeal']=25*((Colombia.Talla/100)**2)
Colombia['ExcesoPeso']=Colombia.PesoInicial-Colombia['PesoIdeal']
position=Colombia.columns.to_list().index('Talla')+1
Colombia=reorder_columns(Colombia, ['PesoIdeal','ExcesoPeso'], position)

### Trombosis
se crean dummies por riesgo de trombosis

In [ ]:
#riesgo de trombosis en el control 0 (cirugia)
Colombia=Colombia.join(pd.get_dummies(Colombia['risk_thrombosis'],prefix="RdTV"))
position=Colombia.columns.to_list().index('risk_thrombosis')+1
Colombia=reorder_columns(Colombia, ['RdTV_Alto','RdTV_Medio','RdTV_Bajo'], position)
Colombia.rename(columns={'RdTV_Alto':'RdTV_Alto_C0','RdTV_Medio':'RdTV_Medio_C0','RdTV_Bajo':'RdTV_Bajo_C0'},inplace=True)
Colombia=Colombia.drop('risk_thrombosis',axis=1)

In [ ]:
#riesgo de trombosis en el 1 control
Colombia=Colombia.replace({'risk_thrombosis.1': 9}, np.nan)
Colombia=Colombia.join(pd.get_dummies(Colombia['risk_thrombosis.1'],prefix="RdTV"))
position=Colombia.columns.to_list().index('risk_thrombosis.1')+1
Colombia=reorder_columns(Colombia, ['RdTV_Alto','RdTV_Medio','RdTV_Bajo'], position)
Colombia.rename(columns={'RdTV_Alto':'RdTV_Alto_C1','RdTV_Medio':'RdTV_Medio_C1','RdTV_Bajo':'RdTV_Bajo_C1'},inplace=True)
Colombia=Colombia.drop('risk_thrombosis.1',axis=1)

## Creación de variables de los controles

Se renombran las variables utilizadas en los controles para que todas queden en el mismo formato,y se crea la lista de comorbilidades <br>
Formato: 'variable'_C#

Se crea la función rename_control, que renombra las variables de los controles con el formato dispuesto. <br>
<font size="1"> La función coge un diccionario que tiene el nombre de la variable actual y el nuevo nombre. recorre todas las variables de los controles que tengan ese nombre y las renombra con el nuevo nombre y el formato dispuesto. <br> si hay una variable que no está en un control se la pone en la condicion, con la lista de los controles en los que no aparece.<font>

In [ ]:
def rename_control(dataFrame,new_var_names,condition={}):
    """rename the control variables of the dataframe.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        new_var_names (dict): 
            keys() = variable names in dataframe to rename
            values()= new names of the variables
        condition (dict): 
            keys()= variable names that are included in the condition, for not being in an specific control 
            values()= number of the control to be omitted
    Returns:
        pd.DataFrame: re-assigned dataframe
    """

    var_names=list(new_var_names.keys()) #create a list of old variable names

    if len(condition)>=1: # if ther is a variable in condition, remove that variable from var_names list, for renaming in other condition
        for c in condition.keys():
            var_names.remove(c)

    vars_rename={}

    for v in var_names: #for each var in var_names rename it
        i=0
        for var in dataFrame.columns:
            if v==var or v+'.' in var:
                vars_rename[var]=new_var_names[v]+'_C'+str(i)
                i=i+1

    if len(condition)>=1:

        for c,ctr in condition.items():
            i=0
            for var in Colombia.columns:
                if c==var or c+'.' in var:
                    if i in ctr:
                        i=i+1
                    vars_rename[var]=new_var_names[c]+'_C'+str(i)
                    i=i+1
    
    dataFrame.rename(columns=vars_rename,inplace=True)
    return dataFrame

Se renombra la variable si asistió o no al control

In [ ]:
Colombia=rename_control(dataFrame=Colombia,new_var_names={'no_control':'NoAsistio'},condition={'no_control':[0]})

### Comorbilidades
Se renombran las variables de los controles

In [ ]:
vars_control={'date_control':'date','weight':'Peso','diabetes_type2':'DT2','hypertension':'Hipert','depression':'Depre','sleep_apnea':'AdS','dyslipidemia':'Dislip','muscle_bone_pain':'DoM','erge':'ERGE','malnutrition':'MalN'}
Colombia=rename_control(Colombia,vars_control,condition={'date_control':[0]})
del vars_control
Colombia['date_C0']=Colombia['FechaCirugia']
position=Colombia.columns.to_list().index('DT2_C0')-1
Colombia=reorder_columns(Colombia, ['date_C0','Peso_C0'], position)
Colombia['PesoCiru']=Colombia['Peso_C0']
position=Colombia.columns.to_list().index('FechaCirugia')+1
Colombia=reorder_columns(Colombia, ['PesoCiru'], position)

Para cada control crear una variable de IMC

In [ ]:
for i in range(8):
    Colombia['IMC_C'+str(i)]=np.nan
    for j in range(len(Colombia)):
        Colombia.iloc[j,Colombia.columns.get_loc('IMC_C'+str(i))]=Colombia.iloc[j,Colombia.columns.get_loc('IMC_C'+str(i))]/((Colombia.iloc[18,Colombia.columns.get_loc('Talla')]/100)**2)       

Para cada control crear una variable de IMC por rango

In [ ]:
bins = [0, 30, 35, 40, 45, 50, 150]
labels = ['<29,9','30-34,9', '35-39,9', '40-44,9', '45-49,9','>50']
for i in range(8):
    Colombia['IMCR_C'+str(i)] = pd.cut(Colombia['IMC_C'+str(i)], bins, labels = labels,include_lowest = True)
del bins, labels

In [ ]:
for i in range(8):
        position=Colombia.columns.to_list().index('Peso_C'+str(i))+1
        Colombia=reorder_columns(Colombia, ['IMC_C'+str(i),'IMCR_C'+str(i)], position)

Para cada control se hace la suma de comorbilidades que tiene el paciente

Se crea una lista de comorbilidades <br>

In [ ]:
comorbilidades=['DT2','Hipert','Depre','AdS','Dislip','DoM','ERGE','MalN']

In [ ]:
#añadir riesgo trombosis para c0 y c1
for i in range(8):
    ComorbControl=[comorb+'_C'+str(i) for comorb in comorbilidades] 
    if 0<=i<1:
        RT=['RdTV_Alto_C0','RdTV_Medio_C0','RdTV_Bajo_C0','RdTV_Alto_C1','RdTV_Medio_C1','RdTV_Bajo_C1']
        ComorbControl.extend(RT)
    Colombia['comor_C'+str(i)]=Colombia[ComorbControl].sum(axis=1,skipna=True)
    position=Colombia.columns.to_list().index('IMCR_C'+str(i))+1
    Colombia=reorder_columns(Colombia, ['comor_C'+str(i)], position)
del ComorbControl, RT

Se crea la diferencia de días entre los controles

In [ ]:
for i in range(8):
    Colombia['date_C'+str(i)]=Colombia['date_C'+str(i)].astype('datetime64[ns]')
    if i!=0:
        Colombia['dias_C'+str(i)]=_Date-Colombia['date_C'+str(i)]
        position=Colombia.columns.to_list().index('date_C'+str(i))+1
        Colombia=reorder_columns(Colombia, ['dias_C'+str(i)], position)

Se crean las variables de perdida de peso entre controles y la cirugía: niveles, porcentage y EWL

In [ ]:
for i in range(8):
    if i!=0:
        Colombia['Ppeso_C'+str(i)]=Colombia['Peso_C0']- Colombia['Peso_C'+str(i)]
        Colombia['EWL_C'+str(i)]=(Colombia['Ppeso_C'+str(i)]/Colombia['ExcesoPeso']*100)
        Colombia['PPP_C'+str(i)]=((Colombia['Ppeso_C'+str(i)]/Colombia['Peso_C0'])-1)*100
        position=Colombia.columns.to_list().index('Peso_C'+str(i))+1
        Colombia=reorder_columns(Colombia, ['Ppeso_C'+str(i),'EWL_C'+str(i),'PPP_C'+str(i)], position)

### Complicaciones
Se renombran las variables de las posibles complicaciones

In [ ]:
complicaciones={'leak':'Fuga','bleeding_transfusion':'ScT','bleeding_reintervention':'ScR','obstruction':'Obstr','deceased':'PaF','thrombosis':'TrV','thromboembolism':'TrP','uci':'TnpUCI','readm':'ReH','no_complications':'NhC','fistula':'Fist','reintervention':'Reint'}
condiciones={'leak':[0],'bleeding_transfusion':[0],'bleeding_reintervention':[0],'obstruction':[0],'deceased':[0],'thrombosis':[0,3],'thromboembolism':[0,3],'uci':[0],'readm':[0],'no_complications':[0],'fistula':[0,1],'reintervention':[0,1]}
Colombia=rename_control(Colombia,complicaciones,condition=condiciones)

In [ ]:
Variables=Colombia.columns.to_list()

### Medicamentos
Se renombran las variables de los medicamentos

In [ ]:
medicamentos={'diabetes_medication':'mDT2', 'diabetes_oral':'moDT2', 'diabetes_oral_num':'moDT2num', 'diabetes_insulin':'iDT2','hypertension_medication':'mHipert', 'depression_medication':'mDepre', 'dyslipidemia_medication':'mDislip', 'muscle_bone_pain_medication':'mDoM', 'erge_medication':'mERGE','hypoglycemic_medication':'mHipo','hypertension_medication_num':'mHiper','sleep_apnea_polysomnographic':'AdS_poli','cpap_machine':'AdS_CPAP'}
Colombia=rename_control(Colombia,medicamentos)
del medicamentos

### Fecha Fallecimieinto
Se renombra la fecha de fallecminento (hay 1 por control) y se vuelve tipo fecha

In [ ]:
Colombia=rename_control(Colombia,new_var_names={'date_decease':'PaF_fecha'},condition={'date_decease':[0]})
for var in Colombia.columns:
    if 'PaF_fecha' in var:
        Colombia[var]=Colombia[var].astype('datetime64[ns]')

### Fechas
Se renombran y vuelven tipo fecha, las fechas de ingreso y salida a UCI o al Hospital

In [ ]:
otherDates={'date_uci_adm':'TnpUCI_ingreso','date_uci_exit':'TnpUCI_salida','date_readm_adm':'ReH_ingreso','date_readm_exit':'ReH_salida'}
condiciones={'date_uci_adm':[0],'date_uci_exit':[0],'date_readm_adm':[0],'date_readm_exit':[0]}
Colombia=rename_control(Colombia,otherDates,condiciones)
for d in otherDates.values():
    for var in Colombia.columns:
        if d in var:
            Colombia[var]=Colombia[var].astype('datetime64[ns]')
del otherDates

### Numero de controles a los que asistio la persona

In [ ]:
Colombia['controles']=Colombia[['date_C1', 'date_C2', 'date_C3', 'date_C4', 'date_C5', 'date_C6','date_C7']].notna().sum(axis=1,skipna=True)
Colombia['controles'].mask(Colombia['Dias'] < timedelta(days=41), 99, inplace=True)

## Recodificar algunas variables del procedimiento
Se eliminan las variables 'OtrosProcelegacy' y 'CausaConverLegacy'

In [ ]:
Colombia['BiopsiaHep'].mask(Colombia['OtrosProcelegacy'] == 'Biopsia hepática', 1, inplace=True)
Colombia['Colecistec'].mask(Colombia['OtrosProcelegacy'] == 'Colecistectomía', 1, inplace=True)
Colombia['CorrecHiato'].mask(Colombia['OtrosProcelegacy'] == 'Corrección de Hiato', 1, inplace=True)
Colombia['NsrO'].mask(Colombia['OtrosProcelegacy'] == 'No se realizó otro procedimiento', 1, inplace=True)
Colombia['OtrosProce'].mask(Colombia['OtrosProcelegacy'] == 'Otra', 1, inplace=True)

Colombia['Colecistec'].mask(Colombia['CausaConverLegacy'] == 'Reganancia de peso', 1, inplace=True)
Colombia['Malnutricion'].mask(Colombia['CausaConverLegacy'] == 'Desnutrición', 1, inplace=True)
Colombia['Reflujo'].mask(Colombia['CausaConverLegacy'] == 'Reflujo - Hernia', 1, inplace=True)
Colombia['Hipoglicemia'].mask(Colombia['CausaConverLegacy'] == 'Hipoglicemia', 1, inplace=True)

Colombia=Colombia.drop(labels=['OtrosProcelegacy','CausaConverLegacy'],axis=1)

Se crea una variable llamada IMC Bajo

In [ ]:
Colombia['IMC_bajo']=np.where(Colombia['IMC_C0'] < 40, 1, 2) #& Colombia['IMC_C0']!=np.nan

Se elminan algunas observaciones que no deberían estar en el análisis

In [ ]:
#Elimino observaciones que tengan menos de 15 años y algunas fechas:
Colombia.drop(Colombia[Colombia['EdadCiru'] < 15].index, inplace = True)
Colombia.drop(Colombia[Colombia['FechaCirugia']>_Date].index, inplace = True)

for m in [1,5,6,8,9]:
    for d in range(1,32):
        if m==6 or m==9:
            if d==31:
                continue
        Colombia.drop(Colombia[Colombia['FechaCirugia']==date(2014,month=m,day=d)].index, inplace = True)

# Exportar Base de Datos
Exportar la base de datos de colombia para la su uso en otros scripts

In [ ]:
Colombia.to_csv(Path_working+'Colombia_preliminar.csv') 

## Tablas de verificación
Exportar las tablas de verificación a excel 

In [ ]:
TablaVerificacion=pd.crosstab(Colombia['Grupo'], [pd.DatetimeIndex(Colombia['FechaCirugia']).year,pd.DatetimeIndex(Colombia['FechaCirugia']).month],colnames=['Año','Mes'],margins=True)
TablaVerificacion_xlsx_path=Path_out_xlsx+'TablaVerificacion'+_Date.strftime("%mm%Y")+'.xlsx'
TablaVerificacion.to_excel(TablaVerificacion_xlsx_path)

In [ ]:
TablaControles=pd.DataFrame(data=(Colombia.groupby('Grupo')['controles'].mean(),Colombia.groupby('Grupo')['controles'].count()),columns=list(Colombia['Grupo'].unique()),index=['Media','Recuento']).T
TC_total=pd.DataFrame(TablaControles.sum(),columns=['Total']).T
pd.concat([TablaControles,TC_total])
TablaControles_xlsx_path=Path_out_xlsx+'TablaControles'+_Date.strftime("%mm%Y")+'.xlsx'
TablaControles.to_excel(TablaControles_xlsx_path)